<a href="https://colab.research.google.com/github/Priyanka-3008/MVP-Assignment/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
import yfinance as yf
data=yf.download('AAPL',start='2008-01-01',end='2024-08-01')
print(data.head())

[*********************100%%**********************]  1 of 1 completed

                Open      High       Low     Close  Adj Close      Volume
Date                                                                     
2008-01-02  7.116786  7.152143  6.876786  6.958571   5.876342  1079178800
2008-01-03  6.978929  7.049643  6.881786  6.961786   5.879056   842066400
2008-01-04  6.837500  6.892857  6.388929  6.430357   5.430275  1455832000
2008-01-07  6.473214  6.557143  6.079643  6.344286   5.357594  2072193200
2008-01-08  6.433571  6.516429  6.100000  6.116071   5.164872  1523816000


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4173 entries, 2008-01-02 to 2024-07-31
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       4173 non-null   float64
 1   High       4173 non-null   float64
 2   Low        4173 non-null   float64
 3   Close      4173 non-null   float64
 4   Adj Close  4173 non-null   float64
 5   Volume     4173 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 228.2 KB


In [4]:
data = data['Close'].resample('M').last().to_frame()
data['Prediction'] = data[['Close']].shift(-6)
data.dropna(inplace=True)

In [5]:
data['SMA_50'] = data['Close'].rolling(window=50).mean()
data['EMA_20'] = data['Close'].ewm(span=20).mean()

delta = data['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
data['RSI'] = 100 - (100 / (1 + rs))


In [6]:
data['Close_Lag1'] = data['Close'].shift(1)
data['Close_Lag2'] = data['Close'].shift(2)
data['Close_Lag180'] = data['Close'].shift(180)

In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_data= scaler.fit_transform(data['Close'].values.reshape(-1, 1))

In [8]:
training_data_len = int(len(scaled_data) * 0.8)
train_data = scaled_data[:training_data_len, :]
test_data = scaled_data[training_data_len-6:, :]

In [9]:
def create_dataset(dataset, look_back=6):
    X, Y = [], []
    for i in range(look_back, len(dataset)):
        X.append(dataset[i-look_back:i, 0])
        Y.append(dataset[i, 0])
    return np.array(X), np.array(Y)

In [10]:
x_train, y_train = create_dataset(train_data)
x_test, y_test = create_dataset(test_data)

In [11]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [12]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(12))
model.add(Dense(6))
model.add(Dense(1))

In [17]:
model.compile(optimizer='adam', loss='mean_squared_error',metrics=['r2_score'])
model.fit(x_train, y_train, batch_size=20, epochs=10)

Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.0011 - r2_score: 0.8896
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0014 - r2_score: 0.9136 
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 8.8767e-04 - r2_score: 0.9342
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 9.3596e-04 - r2_score: 0.9357
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 7.9985e-04 - r2_score: 0.9466
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0015 - r2_score: 0.9051
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0011 - r2_score: 0.9308    
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0013 - r2_score: 0.9002
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 5.9075e-04 - r2_score: 0.9370
Epoch 10/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 7.1336e-04 - r2_score: 0.9393


In [18]:
model.compile(optimizer='adam', loss='mean_squared_error',metrics=['r2_score'])
model.fit(x_train, y_train, batch_size=20, epochs=10)

Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0015 - r2_score: 0.8828
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0014 - r2_score: 0.9033
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0012 - r2_score: 0.9225
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0013 - r2_score: 0.9116
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 9.1251e-04 - r2_score: 0.9241
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 9.1716e-04 - r2_score: 0.9179
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 7.4529e-04 - r2_score: 0.9500
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0011 - r2_score: 0.9244
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 8.8229e-04 - r2_score: 0.9411
Epoch 10/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 8.5945e-04 - r2_score: 0.9262


In [15]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
print('Root Mean Squared Error:', rmse)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 318ms/step
Root Mean Squared Error: 205.92372245154382


In [16]:
last_6_months = scaled_data[-6:]
last_6_months = np.reshape(last_6_months, (1, last_6_months.shape[0], 1))
future_predictions = model.predict(last_6_months)
future_predictions = scaler.inverse_transform(future_predictions)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


In [19]:
print(future_predictions)

[[260.5568]]
